### VGG16 Implementation code

In [52]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [53]:
IMAGE_SIZE = [224,224]

In [54]:
# Give dataset path
train_path = 'data/train'
test_path = 'data/test'

In [55]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)

In [56]:
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [57]:
# useful for getting number of classes 
folders = glob('data/train/*')
print(len(folders))

2


In [58]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation= 'softmax')(x)
model = Model(inputs=vgg.input, outputs = prediction)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [59]:
from keras import optimizers

sgd = optimizers.SGD(lr = 0.01, decay=1e-6, momentum= 0.9)
model.compile(loss='binary_crossentropy',
             optimizer = sgd,
             metrics = ['accuracy'])

In [60]:
datagen = ImageDataGenerator(rescale= 1.0/255.0)
train_set = datagen.flow_from_directory('data/train/', batch_size=64 , target_size=(224,224))
test_set = datagen.flow_from_directory('data/test/', batch_size=64 , target_size=(224,224))

Found 10 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


In [61]:
# Data Augmentation
train_datagen = ImageDataGenerator(preprocessing_function= preprocess_input,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

In [62]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                 rotation_range=40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 shear_range =0.2,
                                 zoom_range=0.2,
                                 horizontal_flip = True,
                                 fill_mode= 'nearest')

In [102]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau


#num_epochs = 1000
#num_batch_size = 32

checkpoint = ModelCheckpoint(filepath= 'mymodel.h5',
                            verbose= 1, save_best_only=True)

callbacks = [checkpoint]

start = datetime.now()

r = model.fit_generator(train_set,
                   validation_data = test_set,
                   epochs=20,
                   steps_per_epoch = 5,
                   validation_steps = 32,
                   callbacks= callbacks, verbose = 1)

duration = datetime.now() - start

print('Training completed in time: ', duration)

In [82]:
import cv2

In [83]:
from keras.models import load_model
model = load_model('mymodel.h5')
img = cv2.imread('data/test/car/1.jpg')
dim = (224,224)
img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
x = image.img_to_array(img)
x= np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)

In [84]:
preds

array([[1., 0.]], dtype=float32)

In [103]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')



In [104]:
# accuracies
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')